In [1]:
import keras
from keras.models import Sequential, Model, load_model
from keras.layers import Dense, Conv2D, Input, Lambda, Flatten, Reshape, MaxPooling2D, Multiply, Activation, Add, Concatenate
from keras.applications.vgg16 import VGG16 
from keras.optimizers import Adam, SGD
from keras.callbacks import EarlyStopping
import keras.backend as K
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
import os
import cv2
import h5py
import matplotlib.pyplot as plt
from google.colab import drive

Using TensorFlow backend.


In [0]:
from numpy.random import seed
seed(1)
from tensorflow import set_random_seed
set_random_seed(2)

In [0]:
from tensorflow.python.client import device_lib
assert len(K.tensorflow_backend._get_available_gpus()) > 0
assert 'GPU' in str(device_lib.list_local_devices())

In [4]:
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [5]:
%cd "drive/My Drive/BTP/code"
%ls

/content/drive/My Drive/BTP/code
Attention_train_v1.ipynb  multi-head_attention_mnist_v1.ipynb
Bateaux_code/             multi-head_attention_mnist_v2.ipynb
bateaux_metrics_v1.py     multi-head_attention_mnist_v3.ipynb
bateaux_metrics_v2.py     playground.ipynb
dataset_creation/         __pycache__/
evaluate_models.ipynb     siamese.ipynb
extra/                    Untitled0.ipynb
__init__.py


In [0]:
from bateaux_metrics_v2 import *

In [0]:
parent_dir = os.path.dirname(os.getcwd())
train_set_file = os.path.join(parent_dir, 'sets_labels', 'single-scene', 'train_set.hdf5')
train_GT_file = os.path.join(parent_dir, 'sets_labels', 'single-scene', 'train_GT.csv')
test_set_file = os.path.join(parent_dir, 'sets_labels', 'single-scene', 'test_set.hdf5')
test_GT_file = os.path.join(parent_dir, 'sets_labels', 'single-scene', 'test_GT.csv')

weight_file = os.path.join(parent_dir, 'models', 'Attention', '1-scene', 'att_model_v8-2.hdf5')
info_file = os.path.join(parent_dir, 'models', 'Attention', '1-scene', 'att_model_v8-2.txt')

In [0]:
K.clear_session()

In [0]:
img_width = 128
img_height = 128
channels = 3

In [0]:
def main_attention(x, l, d, dv, nv):  #x, l=6*6, d=64*3 , dv=8*3, nv = 8
    v = Dense(d, activation = 'relu')(x)
    k = Dense(d, activation = 'relu')(x)
    q = Dense(d, activation = 'relu')(x)
   
    v = Reshape([l,nv,dv])(v)  
    k = Reshape([l,nv,dv])(k)  
    q = Reshape([l,nv,dv])(q)
   
    new = Multiply()([q,k])
    new = Activation('softmax')(new)
   
    new1 = Multiply()([new,v])
    new1 = Reshape([l,d])(new1)
    out = Add()([new1,x])
   
    return out

In [0]:
def multi_head(l, d, dv, nv): #l=6*6, d=64*3 , dv=8*3, nv = 8
   
    input_img = Input(shape = (l,d))
   
    out_1 = main_attention(input_img, l=l, d=d, dv=dv, nv=nv)
    out_2 = main_attention(input_img, l=l, d=d, dv=dv, nv=nv)
    out = Concatenate(axis = 2)([out_1,out_2])
    att_model = Model(input = input_img, output = out)
    #print(att_model.summary())
    return att_model

In [13]:
inp = Input((img_width, img_height, channels), name='input_layer')
x = Conv2D(64, (3,3),activation='relu', padding='same')(inp)
x = Conv2D(64, (3,3),activation='relu', padding='same')(x)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Conv2D(128, (3,3),activation='relu', padding='same')(x)
x = Conv2D(128, (3,3),activation='relu', padding='same')(x)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Conv2D(256, (3,3),activation='relu', padding='same')(x)
x = Conv2D(256, (3,3),activation='relu', padding='same')(x)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Conv2D(512, (3,3),activation='relu', padding='same')(x)
x = Conv2D(512, (3,3),activation='relu', padding='same')(x)
x = MaxPooling2D(pool_size=(2, 2))(x)

x = Reshape([8*8, 512])(x) #x.shape[0] = 8?
att = multi_head(l=8*8, d=512 , dv=8*8, nv = 8)
x = att(x)
x = Dense(32, activation = 'relu')(x)
x = Flatten()(x)
out = Dense(6, activation='relu')(x)
model = Model(input = inp, output = out)
print(model.summary())




Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_layer (InputLayer)     (None, 128, 128, 3)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 128, 128, 64)      1792      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 128, 128, 64)      36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 64, 64, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 64, 64, 128)       73856     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 64, 64, 128)       147584    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 32, 32, 128)       0

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("co...)`
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


In [0]:
y_train = np.loadtxt(train_GT_file, dtype=np.float32)
h5_reader = h5py.File(train_set_file, 'r')

print(list(h5_reader.keys()))

x_train = np.array(h5_reader['train_img'], dtype=np.float32, copy=False) / 255
#x_train = np.array(x_train)
print(x_train.shape)
h5_reader.close()
# print(list(h5_reader.keys()))
# x_train = np.array(h5_reader['train_img'])/255
# # x_train = np.array(x_train)
# # x_train = x_train/255
# h5_reader.close()

['train_img']
(26399, 128, 128, 3)


In [0]:
from sklearn.model_selection import train_test_split
X_train, X_val, Y_train, Y_val = train_test_split(x_train, y_train, test_size=0.1, random_state=12)

In [0]:
print(X_train.shape)
print(Y_train.shape)
print(X_val.shape)
print(Y_val.shape)

(23759, 128, 128, 3)
(23759, 6)
(2640, 128, 128, 3)
(2640, 6)


In [0]:
epochs = 100
batch_size = 128
epochs_to_wait_for_improve = 4
steps_per_epoch = X_train.shape[0] // batch_size
print(steps_per_epoch)

185


In [0]:
#adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
lr = 0.01
momentum = 0.05
nesterov = True
optimizer = SGD(lr=lr, momentum=momentum, nesterov=nesterov)
model.compile(optimizer=optimizer, loss=euclidean_loss(beta=beta), metrics=[euclidean_dist, x_error, y_error, z_error])
early_stopping_callback = EarlyStopping(monitor='val_euclidean_dist', patience=epochs_to_wait_for_improve, restore_best_weights=True)

In [0]:
img_gen = ImageDataGenerator()

In [0]:
model.fit_generator(img_gen.flow(X_train, Y_train, batch_size=batch_size),
            steps_per_epoch=steps_per_epoch, validation_data=(X_val, Y_val),
            epochs=epochs, callbacks=[early_stopping_callback])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/100


185/185 [==============================] - 52s 282ms/step - loss: 0.4364 - euclidean_dist: 0.0273 - x_error: 0.0250 - y_error: 0.0236 - z_error: 0.0251 - val_loss: 0.4191 - val_euclidean_dist: 0.0266 - val_x_error: 0.0249 - val_y_error: 0.0227 - val_z_error: 0.0246
Epoch 2/100
185/185 [==============================] - 45s 244ms/step - loss: 0.4118 - euclidean_dist: 0.0267 - x_error: 0.0249 - y_error: 0.0219 - z_error: 0.0251 - val_loss: 0.4174 - val_euclidean_dist: 0.0260 - val_x_error: 0.0249 - val_y_error: 0.0206 - val_z_error: 0.0246
Epoch 3/100
185/185 [==============================] - 45s 244ms/step - loss: 0.4204 - euclidean_dist: 0.0261 - x_error: 0.0249 - y_error: 0.0196 - z_error: 0.0251 - val_loss: 0.4182 - val_euclidean_dist: 0.0257 - val_x_error: 0.0249 - val_y_error: 0.0186 - val_z_error: 0.0246
Epoch 4/100
185/185 [==============================] - 45s 244ms/ste

In [0]:
model.save(weight_file)
print('Saved trained model at %s ' % weight_file)

Saved trained model at /content/drive/My Drive/BTP/models/Attention/att_model_v8-2.hdf5 


In [0]:
h5_reader = h5py.File(test_set_file, 'r')
#print(h5_reader.keys())
x_test = h5_reader['test_img']
x_test = np.array(x_test)
h5_reader.close()
x_test = x_test.astype('float32')
x_test /= 255

In [0]:
y_test = np.loadtxt(test_GT_file, dtype=np.float32)

In [0]:
scores = model.evaluate(x_test, y_test, verbose=1)
metric_names = model.metrics_names

6600/6600 [==============================] - 5s 793us/step


In [0]:
for metric, val in zip(metric_names, scores):
  print("{} : {}".format(metric, val))

loss : 0.40950297240054967
euclidean_dist : 0.02553656424085299
x_error : 0.024965304244648327
y_error : 0.017031753957271575
z_error : 0.02507539244763779


In [0]:
file_obj = open(info_file, 'w')
file_obj.write("MODEL SUMMARY\n")
#summary = model.summary()
model.summary(print_fn=lambda x: file_obj.write(x + '\n'))
file_obj.write("\n")
#file_obj.write("Optimizer : Adam - batch_size = {}\n\n".format(batch_size))
file_obj.write("Optimizer : SGD - lr = {}, momentum = {}, batch_size = {}\n\n".format(lr, momentum, batch_size))

file_obj.write("\n\nPerformance")
for metric, val in zip(metric_names, scores):
  file_obj.write("{} : {}\n".format(metric, val))

file_obj.write("\n\nFILE PATHS\n")
file_obj.write("Training set : {}\n".format(train_set_file))
file_obj.write("Testing set : {}\n".format(test_set_file))
file_obj.write("Weight file : {}\n".format(weight_file))
file_obj.close()